In [63]:
import json

f = open('NFA.json')
NFA = json.load(f)
f.close()
states = NFA.keys()

for state in states:
    print(state , NFA.get(state) , sep =" : ")

StartingState : S0
S0 : {'IsTerminating': False, 'epsilon': ['S1', 'S7']}
S1 : {'IsTerminating': False, 'epsilon': ['S2', 'S4']}
S2 : {'IsTerminating': False, 'a': ['S3']}
S3 : {'IsTerminating': False, 'epsilon': ['S6']}
S4 : {'IsTerminating': False, 'b': ['S5']}
S5 : {'IsTerminating': False, 'epsilon': ['S6']}
S6 : {'IsTerminating': False, 'epsilon': ['S1', 'S7']}
S7 : {'IsTerminating': False, 'a': ['S8']}
S8 : {'IsTerminating': False, 'b': ['S9']}
S9 : {'IsTerminating': False, 'b': ['S10']}
S10 : {'IsTerminating': True}


In [64]:
def epsilon_closure_state_rec(NFA,state,visited):
    reachable = [state]
    visited.append(state)
    for transition in NFA.get(state):
        if transition == 'epsilon':
            reachable.extend((NFA.get(state).get(transition)))
            for state in reachable:
                if state not in visited:
                    visited.append(state)
                    reachable.extend(epsilon_closure_state_rec(NFA,state,visited))
    return list(set(reachable))


In [65]:
def epsilon_closure_state(NFA,state):
    visited = []
    return epsilon_closure_state_rec(NFA,state,visited)


In [66]:
#test epsilon_closure_state
for state in states:
    print(state , epsilon_closure_state(NFA,state) , sep =" : ")

StartingState : ['StartingState']
S0 : ['S0', 'S2', 'S7', 'S4', 'S1']
S1 : ['S1', 'S2', 'S4']
S2 : ['S2']
S3 : ['S2', 'S7', 'S4', 'S6', 'S3', 'S1']
S4 : ['S4']
S5 : ['S2', 'S5', 'S7', 'S4', 'S6', 'S1']
S6 : ['S2', 'S7', 'S4', 'S6', 'S1']
S7 : ['S7']
S8 : ['S8']
S9 : ['S9']
S10 : ['S10']


In [67]:
def epsilon_closure_states(NFA,states):
    reachable = []
    visited = []
    for state in states:
        reachable.extend(epsilon_closure_state(NFA,state))
    return list(set(reachable))

In [68]:
#test epsilon closure of states
sorted = lambda x: x.sort()
ans = epsilon_closure_states(NFA,['S1','S4','S8'])
sorted(ans)
print(ans)

['S1', 'S2', 'S4', 'S8']


In [69]:
def move(NFA,states,symbol):
    reachable = []
    for state in states:
        if symbol in NFA.get(state):
            reachable.extend(NFA.get(state).get(symbol))
    return list(set(reachable))

In [70]:
#test move
print(epsilon_closure_states(NFA,['S5','S9']))

['S2', 'S5', 'S9', 'S7', 'S4', 'S6', 'S1']


In [ ]:
def DFA_state(states , IsTerminating):
    return {'states':states , 'IsTerminating':IsTerminating , 'marked' : False}

def Extract_NFA_transitions(NFA):
    transitions = []
    for state in NFA:
        if state != "StartingState":
            for transition in NFA.get(state):
                if transition != 'epsilon' and transition != 'IsTerminating' and transition not in transitions:
                    transitions.append(transition)
    return transitions

def unique_state_set(DFA , U):
    for state in DFA.keys():
        if state != "StartingState" and DFA.get(state).get("states") == U:
            return state
    return None

def get_NFA_Terminating_states(NFA):
    terminating_states = set()
    for state in NFA:
        if state != "StartingState" and NFA.get(state).get("IsTerminating") == True:
            terminating_states.add(state)
    return terminating_states

def get_first_unmarked_state(DFA):
    for state in DFA:
        if state != "StartingState" and DFA.get(state).get("marked") == False:
            return state
    return None

In [76]:
def NFA_to_DFA(NFA):
    last_DFA_State_symbol = 65
    terminating_states = get_NFA_Terminating_states(NFA)
    DFA = dict()
    DFA["StartingState"] = chr(last_DFA_State_symbol)
    
    U = epsilon_closure_state(NFA, NFA.get("StartingState"))
    if len(list(set(U).intersection(terminating_states))) > 0:
        DFA.update({chr(last_DFA_State_symbol): DFA_state(U,True)})
    else:
        DFA.update({chr(last_DFA_State_symbol): DFA_state(U,False)})
    
    transitions = Extract_NFA_transitions(NFA)
    states = [chr(last_DFA_State_symbol)]
    while False in [DFA.get(state).get("marked") for state in states]:
        for transition in transitions: #for each transition in NFA
            state = get_first_unmarked_state(DFA)
            NFA_states = DFA.get(state).get('states')
            U = epsilon_closure_states(NFA,move(NFA,NFA_states,transition))
            unique_state = unique_state_set(DFA,U)
            if unique_state is None:
                last_DFA_State_symbol += 1
                if len(list(set(U).intersection(terminating_states))) > 0:
                    DFA.update({chr(last_DFA_State_symbol): DFA_state(U,True)})
                else:
                    DFA.update({chr(last_DFA_State_symbol): DFA_state(U,False)})
                states.append(chr(last_DFA_State_symbol))
                DFA.get(state).update({transition : chr(last_DFA_State_symbol)})
            else:
                DFA.get(state).update({transition : unique_state})

        DFA.get(state).update({"marked":True})

    
    #remove the marked attribute
    for state in DFA:
        if state != "StartingState":
            DFA.get(state).pop("marked")
        
    return DFA


    

In [77]:
#test NFA_to_DFA
DFA = NFA_to_DFA(NFA)

# Path: NFA_to_DFA.ipynb
import json
file = open('DFA.json','w')
json.dump(DFA,file)
file.close()